matrix transformation only for perfect file (with path 0)

In [ ]:
import numpy as np
import os
import pandas as pd
from scipy.optimize import minimize

# pymatgen libraries
from pymatgen.core.structure import Structure

# import function from .py file
from get_weirdos_ver3_new_cleaned import get_dx_dz_init, get_sum_weirdos_Li_var

In [ ]:
# Replace with your file path
litype = 2
file_perfect_poscar_48n24_wo_cif = "Li6PS5Cl_type2"

activate_radius = 2
max_mapping_radius = 0.043
max_mapping_radius_48htype2 = 0.070

file_ori_ref_48n24 = f"{file_perfect_poscar_48n24_wo_cif}.cif"
file_perfect_poscar_24 = "Li6PS5Cl_24_mod_2p27291.cif"
file_path_ori_ref_48n24 = f"./perfect_poscar/cif_matrix/ori/{file_ori_ref_48n24}"


In [ ]:
dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init, dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init, dx_24g_init, dz1_24g_init, dz2_24g_init = get_dx_dz_init(file_path_ori_ref_48n24)

dx1_48h_type1, dx2_48h_type1, dz_48h_type1 = dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init
dx1_48h_type2, dx2_48h_type2, dz_48h_type2 = dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init
dx_24g, dz1_24g, dz2_24g = dx_24g_init, dz1_24g_init, dz2_24g_init

max_mapping_radius_init = max_mapping_radius
max_mapping_radius_48htype2_init = max_mapping_radius_48htype2

In [ ]:
def objective_function_var(x, max_mapping_radius, max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_ori_ref_48n24, litype, var_optitype):
    global dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g
    dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g = x
    sum_weirdos_Li = get_sum_weirdos_Li_var(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_ori_ref_48n24, litype, var_optitype)
    # sum_weirdos_Li = get_sum_weirdos_Li_w_radius(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, var_optitype)
    return sum_weirdos_Li

# def objective_function(x, var_optitype):
#     global dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g
#     dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g = x
#     sum_weirdos_Li = get_sum_weirdos_Li(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, var_optitype)
#     # sum_weirdos_Li = get_sum_weirdos_Li_w_radius(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, var_optitype)
#     return sum_weirdos_Li

# def objective_function_w_radius(x, var_optitype):
#     global dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2
#     dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2 = x
#     sum_weirdos_Li = get_sum_weirdos_Li_w_radius(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, var_optitype)
#     return sum_weirdos_Li


def apply_constraint(dx1_48h_type1, dz_48h_type1, dz_48h_type2, dx1_48h_type2, dz1_24g, dz2_24g):
    total_48h_type1 = dx1_48h_type1 + dz_48h_type1 - 1
    total_48h_type2 = dz_48h_type2 - dx1_48h_type2 - 0.5
    total_24g = dz1_24g - dz2_24g
    return total_48h_type1, total_48h_type2, total_24g

my_constraints = ({'type': 'eq', 'fun': apply_constraint})

idx_cif_file = 0
def callback_function_var(x):
    global idx_cif_file
    dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g = x
    sum_weirdos_Li = get_sum_weirdos_Li_var(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_ori_ref_48n24, litype, var_optitype)
    # sum_weirdos_Li = get_sum_weirdos_Li_w_radius(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, var_optitype)
    # print('{0:4d}   {1: 3.5f}   {2: 3.5f}   {3: 3.5f}   {4: 3.5f}   {5: 3.5f}   {6: 3.5f}   {7: 4f}'.format(idx_cif_file, dx1_48h_type1, dx2_48h_type1, dx1_48h_type2, dx2_48h_type2, dx_24g, dz1_24g, get_sum_weirdos_Li(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, var_optitype)))
    print(f'{idx_cif_file:4d}   {dx1_48h_type1}   {dx2_48h_type1}   {dx1_48h_type2}   {dx2_48h_type2}   {dx_24g}   {dz1_24g}   {sum_weirdos_Li}')
    idx_cif_file = idx_cif_file + 1

# def callback_function(x):
#     global idx_cif_file
#     dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g = x
#     sum_weirdos_Li = get_sum_weirdos_Li(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, var_optitype)
#     # sum_weirdos_Li = get_sum_weirdos_Li_w_radius(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, var_optitype)
#     # print('{0:4d}   {1: 3.5f}   {2: 3.5f}   {3: 3.5f}   {4: 3.5f}   {5: 3.5f}   {6: 3.5f}   {7: 4f}'.format(idx_cif_file, dx1_48h_type1, dx2_48h_type1, dx1_48h_type2, dx2_48h_type2, dx_24g, dz1_24g, get_sum_weirdos_Li(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, var_optitype)))
#     print(f'{idx_cif_file:4d}   {dx1_48h_type1}   {dx2_48h_type1}   {dx1_48h_type2}   {dx2_48h_type2}   {dx_24g}   {dz1_24g}   {sum_weirdos_Li}')
#     idx_cif_file = idx_cif_file + 1

# def callback_function_w_radius(x):
#     global idx_cif_file
#     dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2 = x
#     sum_weirdos_Li = get_sum_weirdos_Li_w_radius(dx1_48h_type1, dx2_48h_type1, dz_48h_type1, dx1_48h_type2, dx2_48h_type2, dz_48h_type2, dx_24g, dz1_24g, dz2_24g, max_mapping_radius, max_mapping_radius_48htype2, var_optitype)
#     # print('{0:4d}   {1: 3.5f}   {2: 3.5f}   {3: 3.5f}   {4: 3.5f}   {5: 3.5f}   {6: 3.5f}   {7: 4f}   {8: 3f}   {9: 3f}'.format(idx_cif_file, dx1_48h_type1, dx2_48h_type1, dx1_48h_type2, dx2_48h_type2, dx_24g, dz1_24g, sum_weirdos_Li, max_mapping_radius, max_mapping_radius_48htype2))
#     print(f'{idx_cif_file:4d}   {dx1_48h_type1}   {dx2_48h_type1}   {dx1_48h_type2}   {dx2_48h_type2}   {dx_24g}   {dz1_24g}   {sum_weirdos_Li}   {max_mapping_radius}   {max_mapping_radius_48htype2}')
#     idx_cif_file = idx_cif_file + 1

# print(f'{"Iter":4s}   {"dx1_48h_type1":9s}   {"dx2_48h_type1":9s}   {"dx1_48h_type2":9s}   {"dx2_48h_type2":9s}   {"dx_24g":9s}   {"dz1_24g":9s}   {"sum_weirdos_Li":4s}')
# print(f'{"Iter":4s}   {"dx1_48h_type1":9s}   {"dx2_48h_type1":9s}   {"dx1_48h_type2":9s}   {"dx2_48h_type2":9s}   {"dx_24g":9s}   {"dz1_24g":9s}   {"sum_weirdos_Li":4s}   {"max_mapping_radius":3s}   {"max_mapping_radius_48htype2":3s}')

# initial_guess_w_radius = [dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init, dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init, dx_24g_init, dz1_24g_init, dz2_24g_init]
initial_guess_w_radius = [dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init, dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init, dx_24g_init, dz1_24g_init, dz2_24g_init, max_mapping_radius_init, max_mapping_radius_48htype2_init]
initial_guess = [dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init, dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init, dx_24g_init, dz1_24g_init, dz2_24g_init]

# # Define bounds for dx1 and dz (allowing up to 5 decimal places)
# bounds = [(round(dx1_48h_type1_init, 5), round(dx1_48h_type1_init, 5)),  # Lower and upper bounds for 48h_type1
#           (round(dx2_48h_type1_init, 5), round(dx2_48h_type1_init, 5)),  
#           (round(dz_48h_type1_init, 5), round(dz_48h_type1_init, 5)),    
#           (round(dx1_48h_type2_init, 5), round(dx1_48h_type2_init, 5)),  # Lower and upper bounds for 48h_type2
#           (round(dx2_48h_type2_init, 5), round(dx2_48h_type2_init, 5)),  
#           (round(dz_48h_type2_init, 5), round(dz_48h_type2_init, 5)),    
#           (round(dx_24g_init, 5), round(dx_24g_init, 5)),                          # Lower and upper bounds for 24g
#           (round(dz1_24g_init, 5), round(dz1_24g_init, 5)),  
#           (round(dz2_24g_init, 5), round(dz2_24g_init, 5))]    

# Define bounds_w_radius for dx1 and dz (allowing up to 5 decimal places)
bounds_w_radius = [(round(dx1_48h_type1_init, 5), round(dx1_48h_type1_init, 5)),  # Lower and upper bounds_w_radius for 48h_type1
          (round(dx2_48h_type1_init, 5), round(dx2_48h_type1_init, 5)),  
          (round(dz_48h_type1_init, 5), round(dz_48h_type1_init, 5)),    
          (round(dx1_48h_type2_init, 5), round(dx1_48h_type2_init, 5)),  # Lower and upper bounds_w_radius for 48h_type2
          (round(dx2_48h_type2_init, 5), round(dx2_48h_type2_init, 5)),  
          (round(dz_48h_type2_init, 5), round(dz_48h_type2_init, 5)),    
          (round(dx_24g_init, 5), round(dx_24g_init, 5)),                          # Lower and upper bounds_w_radius for 24g
          (round(dz1_24g_init, 5), round(dz1_24g_init, 5)),  
          (round(dz2_24g_init, 5), round(dz2_24g_init, 5)),
          (round(max_mapping_radius_init, 3), round(max_mapping_radius_init, 3)),                          # Lower and upper bounds_w_radius for 24g
          (round(max_mapping_radius_48htype2_init, 3), round(max_mapping_radius_48htype2_init, 3))]   

# Define bounds_w_radius for dx1 and dz (allowing up to 5 decimal places)
bounds = [(round(dx1_48h_type1_init, 5), round(dx1_48h_type1_init, 5)),  # Lower and upper bounds_w_radius for 48h_type1
          (round(dx2_48h_type1_init, 5), round(dx2_48h_type1_init, 5)),  
          (round(dz_48h_type1_init, 5), round(dz_48h_type1_init, 5)),    
          (round(dx1_48h_type2_init, 5), round(dx1_48h_type2_init, 5)),  # Lower and upper bounds_w_radius for 48h_type2
          (round(dx2_48h_type2_init, 5), round(dx2_48h_type2_init, 5)),  
          (round(dz_48h_type2_init, 5), round(dz_48h_type2_init, 5)),    
          (round(dx_24g_init, 5), round(dx_24g_init, 5)),                          # Lower and upper bounds_w_radius for 24g
          (round(dz1_24g_init, 5), round(dz1_24g_init, 5)),  
          (round(dz2_24g_init, 5), round(dz2_24g_init, 5))]   

# # Define bounds_w_radius for dx1 and dz (allowing up to 5 decimal places)
# bounds_w_radius = [(round(dx1_48h_type1_init, 5), round(dx1_48h_type1_init, 1)),  # Lower and upper bounds_w_radius for 48h_type1
#           (round(dx2_48h_type1_init, 5), round(dx2_48h_type1_init, 1)),  
#           (round(dz_48h_type1_init, 5), round(dz_48h_type1_init, 1)),    
#           (round(dx1_48h_type2_init, 5), round(dx1_48h_type2_init, 1)),  # Lower and upper bounds_w_radius for 48h_type2
#           (round(dx2_48h_type2_init, 5), round(dx2_48h_type2_init, 1)),  
#           (round(dz_48h_type2_init, 5), round(dz_48h_type2_init, 1)),    
#           (round(dx_24g_init, 5), round(dx_24g_init, 1)),                          # Lower and upper bounds_w_radius for 24g
#           (round(dz1_24g_init, 5), round(dz1_24g_init, 1)),  
#           (round(dz2_24g_init, 5), round(dz2_24g_init, 1)),
#           (round(max_mapping_radius_init, 3), round(max_mapping_radius_init, 1)),                          # Lower and upper bounds_w_radius for max_radius
#           (round(max_mapping_radius_48htype2_init, 3), round(max_mapping_radius_48htype2_init, 1))]    

# # Define bounds_w_radius for dx1 and dz (allowing up to 5 decimal places)
# bounds_w_radius = [(round(dx1_48h_type1_init, 1), round(dx1_48h_type1_init, 5)),  # Lower and upper bounds_w_radius for 48h_type1
#           (round(dx2_48h_type1_init, 1), round(dx2_48h_type1_init, 5)),  
#           (round(dz_48h_type1_init, 1), round(dz_48h_type1_init, 5)),    
#           (round(dx1_48h_type2_init, 1), round(dx1_48h_type2_init, 5)),  # Lower and upper bounds_w_radius for 48h_type2
#           (round(dx2_48h_type2_init, 1), round(dx2_48h_type2_init, 5)),  
#           (round(dz_48h_type2_init, 1), round(dz_48h_type2_init, 5)),    
#           (round(dx_24g_init, 1), round(dx_24g_init, 5)),                          # Lower and upper bounds_w_radius for 24g
#           (round(dz1_24g_init, 1), round(dz1_24g_init, 5)),  
#           (round(dz2_24g_init, 1), round(dz2_24g_init, 5)),
#           (round(max_mapping_radius_init, 1), round(max_mapping_radius_init, 3)),                          # Lower and upper bounds_w_radius for max_radius
#           (round(max_mapping_radius_48htype2_init, 1), round(max_mapping_radius_48htype2_init, 3))]    

# # constrained, bounded
# var_optitype = "constrained_bounded"
# results = minimize(objective_function_w_radius, 
#                    initial_guess_w_radius,
#                    args = var_optitype,
#                    method = "Nelder-Mead",
#                    constraints = my_constraints,
#                    bounds = bounds_w_radius,
#                    callback = callback_function_w_radius)

# # constrained
# var_optitype = "constrained"
# results = minimize(objective_function_w_radius, 
#                    initial_guess_w_radius,
#                    args = var_optitype,
#                    method = "Nelder-Mead",
#                    constraints = my_constraints,
#                    callback = callback_function_w_radius)

# constrained_wo_radius
var_optitype = "constrained_wo_radius"
results = minimize(objective_function_var, 
                   initial_guess,
                   args = (max_mapping_radius, max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_ori_ref_48n24, var_optitype),
                   method = "Nelder-Mead",
                   constraints = my_constraints,
                   callback = callback_function_var)

# # constrained_wo_radius, bounded
# var_optitype = "constrained_bounded_wo_radius"
# results = minimize(objective_function, 
#                    initial_guess,
#                    args = var_optitype,
#                    method = "Nelder-Mead",
#                    constraints = my_constraints,
#                    bounds = bounds,
#                    callback = callback_function)

# ## wo constraint
# var_optitype="woconstrains_bounded"
# results = minimize(objective_function_w_radius, 
#                    initial_guess_w_radius,
#                    args = var_optitype,
#                    method = "Nelder-Mead",
#                    bounds = bounds_w_radius,
#                    callback = callback_function_w_radius)

# ## wo constraint, wo bounds_w_radius
# var_optitype="woconstrains_woubounds_w_radius"
# results = minimize(objective_function_w_radius, 
#                    initial_guess_w_radius,
#                    args = var_optitype,
#                    method = "Nelder-Mead",
#                    callback = callback_function_w_radius)


In [ ]:
results